<p>Generally speaking, all the methods learnt in this lecture is pre-ML era methods</p>
<p>More advanced methods include word embeddings and byte-pair encoding</p>

# 2.Set-based Methods

## 2.1 TF-IDF

<p>Term Frequency (TF): The term frequency represents how often a term appears in a document. It represents how important a term is to one document.<\p>
<p>Inverse Document Frequency (IDF): The IDF factor balances out the importance given to terms that appear very frequently across many documents in the corpus. It can help decrease the significance of the words that frequently appear in all documents.<\p>
<p>TF/IDF = TF * IDF<\p>

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Sample text data
documents = [
    'this is the first document',
    'this document is the second document',
    'and this is the third one',
    'is this the first document'
]

# Create the TfidfVectorizer object
vectorizer = TfidfVectorizer()

# Fit and transform the documents
tfidf_matrix = vectorizer.fit_transform(documents)

# Get the feature names
feature_names = vectorizer.get_feature_names_out()

# Convert to dense matrix and then to DataFrame for better visualization
df = pd.DataFrame(tfidf_matrix.todense(), columns=feature_names)

df


,and,document,first,is,one,second,the,third,this
0,0.000000,0.469791,0.580286,0.384085,0.000000,0.000000,0.384085,0.000000,0.384085
1,0.000000,0.687624,0.000000,0.281089,0.000000,0.538648,0.281089,0.000000,0.281089
2,0.511849,0.000000,0.000000,0.267104,0.511849,0.000000,0.267104,0.511849,0.267104
3,0.000000,0.469791,0.580286,0.384085,0.000000,0.000000,0.384085,0.000000,0.384085


In [49]:
import math
import numpy as np
from collections import Counter
import pandas as pd

# Return a Counter object
# input: 'this is the first document'
# output: Counter({'this': 0.2, 'is': 0.2, 'the': 0.2, 'first': 0.2, 'document': 0.2})
def compute_tf(text):
    """Compute term frequency for each term in a document"""
    tf_text = Counter(text.split())
    for term in tf_text:
        tf_text[term] = tf_text[term] / float(len(text.split()))
    return tf_text

# Return a dictionary, storing the {word i : log((1 + N)/(1 + #appearance of word i)) + 1}
def compute_idf(documents):
    """Compute inverse document frequency for each term in the corpus"""
    idf_dict = {}
    N = len(documents)
    
    for doc in documents:
        for word in set(doc.split()):
            idf_dict[word] = idf_dict.get(word, 0) + 1
    
    for word, val in idf_dict.items():
        idf_dict[word] = math.log((1 + N) / (1 + val)) + 1  # sklearn's IDF formula
        
    return idf_dict

def normalize_vector(vector):
    """Apply L2-normalization to a vector"""
    return vector / np.linalg.norm(vector)

def compute_tfidf(documents):
    """Compute TF-IDF score for each term in each document"""
    tf_values = [compute_tf(doc) for doc in documents]
    idf_values = compute_idf(documents)
    
    # Sort feature names
    unique_words = sorted(list(idf_values.keys()))
    
    tfidf_values = []
    for tf in tf_values:
        tfidf = {}
        for word, val in tf.items():
            tfidf[word] = val * idf_values.get(word, 0)
        tfidf_values.append(tfidf)
    
    # Convert to matrix form and apply L2-normalization
    tfidf_matrix = np.zeros((len(documents), len(unique_words)))
    for i, tfidf in enumerate(tfidf_values):
        for j, word in enumerate(unique_words):
            tfidf_matrix[i, j] = tfidf.get(word, 0)
        tfidf_matrix[i, :] = normalize_vector(tfidf_matrix[i, :])
    
    return tfidf_matrix, unique_words

# Sample text data
documents = [
    'this is the first document',
    'this document is the second document',
    'and this is the third one',
    'is this the first document'
]

# Compute TF-IDF values
tfidf_matrix, feature_names = compute_tfidf(documents)

# Convert to DataFrame for better visualization
df2 = pd.DataFrame(tfidf_matrix, columns=feature_names)

# Display the DataFrame
df2


,and,document,first,is,one,second,the,third,this
0,0.000000,0.469791,0.580286,0.384085,0.000000,0.000000,0.384085,0.000000,0.384085
1,0.000000,0.687624,0.000000,0.281089,0.000000,0.538648,0.281089,0.000000,0.281089
2,0.511849,0.000000,0.000000,0.267104,0.511849,0.000000,0.267104,0.511849,0.267104
3,0.000000,0.469791,0.580286,0.384085,0.000000,0.000000,0.384085,0.000000,0.384085


In [14]:
np.absolute(df2-df)<=1e-10

,and,document,first,is,one,second,the,third,this
0,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True


## 2.2 Generalized Jacard Measure

In [39]:
def jaro_winkler(s1, s2):
    # If the strings are equal
    if s1 == s2:
        return 1.0

    # Length of two strings
    len1, len2 = len(s1), len(s2)
    max_dist = int(max(len1, len2) / 2) - 1


    # identifies and records characters in s1 that have a corresponding match in s2 within a window
    match1, match2 = [], []
    count = 0
    for i, l1 in enumerate(s1):
        left, right = max(0, i - max_dist), min(i + max_dist + 1, len2)
        if l1 in s2[left:right]:
            count += 1
            match1.append(l1)
            match2.append(s2[s2.index(l1)])
            # Replace a found character with * to ensure we don't match it again with another character from s1
            s2 = s2[0:s2.index(l1)] + '*' + s2[s2.index(l1) + 1:]

    # If no characters match
    if count == 0:
        return 0.0


    # Compute transpositions
    t = sum([match1[i] != match2[i] for i in range(len(match1))]) / 2.0


    score = ((count / len1) + (count / len2) + ((count - t) / count)) / 3

    # Adjust for Jaro-Winkler, giving a bonus to strings that have a common prefix
    l = min(len([i for i in range(len(match1)) if s1[i] == s2[i]]), 4)
    score += l * 0.1 * (1 - score)

    return score

def generalized_jaccard(set1, set2, threshold=0.5):
    if not set1 or not set2:
        return 0.0

    # Convert to sets if they are lists
    set1, set2 = set(set1), set(set2)

    match_score = 0.0
    match_count = 0

    for element in set1:
        max_score = 0
        for item in set2:
            score = jaro_winkler(element, item)
            if score > threshold:
                max_score = max(max_score, score)
        
        if max_score > 0:
            match_score += max_score
            match_count += 1

    return float(match_score) / float(len(set1) + len(set2) - match_count)

# Test
print(generalized_jaccard(['apple', 'banana'], ['appel', 'banan'], threshold=0.8))


0.9722222222222223


In [31]:
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import csr_matrix
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.optimize import linear_sum_assignment

# Jaccard Similarity
def jaccard_similarity(str1, str2):
    a = set(str1.split())
    b = set(str2.split())
    return len(a.intersection(b)) / float(len(a.union(b)))

# Generalized Jaccard Similarity
def generalized_jaccard_similarity(str1, str2, threshold=0.7):
    tokens_a = str1.split()
    tokens_b = str2.split()

    # Create and fit a single CountVectorizer on both strings
    vectorizer = CountVectorizer().fit(tokens_a + tokens_b)

    # Transform each string using the shared vocabulary
    vec_a = vectorizer.transform(tokens_a).toarray()
    vec_b = vectorizer.transform(tokens_b).toarray()

    # Compute cosine similarity
    sim_matrix = cosine_similarity(vec_a, vec_b)

    # Thresholding
    sim_matrix[sim_matrix < threshold] = 0

    # Maximum-weight bipartite matching
    row_ind, col_ind = linear_sum_assignment(-sim_matrix)
    total_sim = sim_matrix[row_ind, col_ind].sum()

    # Generalized Jaccard formula
    return total_sim / (len(tokens_a) + len(tokens_b) - total_sim)

# Test the functions
str1 = "Energy & Transportation"
str2 = "Transportation, Energy, & Gas"

print("Jaccard Similarity:", jaccard_similarity(str1, str2))
print("Generalized Jaccard Similarity:", generalized_jaccard_similarity(str1, str2))


Jaccard Similarity: 0.16666666666666666
Generalized Jaccard Similarity: 0.4


## 2.3 Soft TF-IDF measure

<p>Firstly, we get a TF-IDF Matrix M. </p>
<p>Soft TF-IDF is used to compute the similarity between document A and B: for every word i in A, find a most similar word j in B(there is a threshold, if no word in B has a similarity score over the threshold, then this term is 0), and add the following term to the overall score  $$TF-IDF(i) * TF-IDF(j) * similarity(i,j)$$</p>
<p> Its physical meaning is the addition of: the importance of word i in document A * the importance of word j in document B * similarity score</p>

In [58]:
def compute_tf_idf_weight(term, document, corpus):
    tf = document.count(term) / len(document)
    idf = len(corpus) / sum([1 for doc in corpus if term in doc])
    return tf * idf

def soft_tf_idf(x, y, corpus, threshold):
    tfidf_vectorizer = TfidfVectorizer(tokenizer=lambda s: s.split()).fit(corpus)
    vocab = tfidf_vectorizer.get_feature_names_out()

    # x_tf and y_tf are tf-idf matrices of vectors x and y
    x_tf = tfidf_vectorizer.transform([x])
    y_tf = tfidf_vectorizer.transform([y])


    soft_score = 0

    for word_x in x.split():
        if word_x in vocab:
            max_sim = 0
            sim_word = ""
            for word_y in y.split():
                if word_y in vocab:
                    sim = jaro_winkler(word_x, word_y)
                    if sim > threshold and sim > max_sim:
                        max_sim = sim
                        sim_word = word_y
            x_idf_index = tfidf_vectorizer.vocabulary_[word_x]
            x_tf_value = x_tf[0, x_idf_index]
            y_idf_index = tfidf_vectorizer.vocabulary_[sim_word] if sim_word in vocab else -1
            y_tf_value = y_tf[0, y_idf_index] if y_idf_index != -1 else 0
            soft_score += x_tf_value * y_tf_value * max_sim

    return soft_score

# Example
corpus = ["apple is good for health", 
          "Apple's headquarter is in California", 
          "Apple is good for helth", 
          "A17 is not as good as expected"]
x = "apple is good for health"
y = "Apple is good for helth"
print(soft_tf_idf(x, y, corpus, 0.75))


0.7683311334449655
